In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
books = pd.read_csv('./dataset/books.csv')
books = books.drop(['image_url', 'small_image_url', 'best_book_id', 'isbn', 'isbn13', 'original_title', 'language_code', 'ratings_count', 'work_ratings_count', 'work_text_reviews_count', 'ratings_1', 'ratings_2', 'ratings_3', 'ratings_4', 'ratings_5', 'book_id', 'work_id', 'books_count'], axis = 1)
books.set_index('id')
books.head()

,id,authors,original_publication_year,title,average_rating
0,1,Suzanne Collins,2008.0,"The Hunger Games (The Hunger Games, #1)",4.34
1,2,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Sorcerer's Stone (Harry P...,4.44
2,3,Stephenie Meyer,2005.0,"Twilight (Twilight, #1)",3.57
3,4,Harper Lee,1960.0,To Kill a Mockingbird,4.25
4,5,F. Scott Fitzgerald,1925.0,The Great Gatsby,3.89


# Content-based Filtering

In [ ]:
content = books[['title', 'authors', 'average_rating']]
content = content.astype('str')
content.head(5)

,title,authors,average_rating
0,"The Hunger Games (The Hunger Games, #1)",Suzanne Collins,4.34
1,Harry Potter and the Sorcerer's Stone (Harry P...,"J.K. Rowling, Mary GrandPré",4.44
2,"Twilight (Twilight, #1)",Stephenie Meyer,3.57
3,To Kill a Mockingbird,Harper Lee,4.25
4,The Great Gatsby,F. Scott Fitzgerald,3.89


In [ ]:
content['content'] = content['title'] + ' ' + content['authors'] + ' ' + content['average_rating']
indices = pd.Series(content.index, index=content['title'])
content.head(5)

,title,authors,average_rating,content
0,"The Hunger Games (The Hunger Games, #1)",Suzanne Collins,4.34,"The Hunger Games (The Hunger Games, #1) Suzann..."
1,Harry Potter and the Sorcerer's Stone (Harry P...,"J.K. Rowling, Mary GrandPré",4.44,Harry Potter and the Sorcerer's Stone (Harry P...
2,"Twilight (Twilight, #1)",Stephenie Meyer,3.57,"Twilight (Twilight, #1) Stephenie Meyer 3.57"
3,To Kill a Mockingbird,Harper Lee,4.25,To Kill a Mockingbird Harper Lee 4.25
4,The Great Gatsby,F. Scott Fitzgerald,3.89,The Great Gatsby F. Scott Fitzgerald 3.89


In [ ]:
#removing stopwords
tfidf = TfidfVectorizer(stop_words='english')

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(content['authors'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape

NameError: name 'TfidfVectorizer' is not defined

In [ ]:
count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(content['content'])
cosine_sim_content = cosine_similarity(count_matrix, count_matrix)

In [ ]:
def get_recommendations(title, cosine_sim=cosine_sim_content):
    idx = indices[title]

    # Get the pairwsie similarity scores of all books with that book
    sim_scores = list(enumerate(cosine_sim_content[idx]))

    # Sort the books based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar books
    sim_scores = sim_scores[1:11]

    # Get the book indices
    book_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar books
    return list(content['title'].iloc[book_indices])

In [ ]:
def book_shows(book):
    for book in book:
        if book != 'nan':
            print(book)

In [ ]:
book_shows(get_recommendations("Harry Potter and the Sorcerer's Stone (Harry Potter, #1)"))

Harry Potter and the Goblet of Fire (Harry Potter, #4)
The Harry Potter Collection 1-4 (Harry Potter, #1-4)
Harry Potter and the Order of the Phoenix (Harry Potter, #5)
Harry Potter and the Chamber of Secrets (Harry Potter, #2)
Harry Potter and the Deathly Hallows (Harry Potter, #7)
Harry Potter and the Half-Blood Prince (Harry Potter, #6)
Harry Potter Boxed Set, Books 1-5 (Harry Potter, #1-5)
Harry Potter and the Prisoner of Azkaban (Harry Potter, #3)
Harry Potter Boxset (Harry Potter, #1-7)
Harry Potter Collection (Harry Potter, #1-6)


# Collaborative Filtering

In [ ]:
reader = Reader()

In [ ]:
ratings = pd.read_csv('./dataset/ratings.csv')
ratings.head(5)

,book_id,user_id,rating
0,1,314,5
1,1,439,3
2,1,588,5
3,1,1169,4
4,1,1185,4


In [ ]:
data = Dataset.load_from_df(ratings[['book_id', 'user_id', 'rating']], reader)
svd = SVD()

In [ ]:
trainset = data.build_full_trainset()
svd.fit(trainset)

In [ ]:
svd.predict(501, 6868, 4)

Prediction(uid=501, iid=6868, r_ui=4, est=3.260060092906961, details={'was_impossible': False})

# Hybrid Filtering
### Menggunakan Content-based Filtering dan Collaborative Filtering

In [ ]:
def convert_int(x):
    try:
        return int(x)
    except:
        return np.nan

In [ ]:
id_map = pd.read_csv('./dataset/books.csv')[['book_id', 'work_id', 'id']]
id_map['book_id'] = id_map['book_id'].apply(convert_int)
id_map = id_map.merge(books[['title', 'book_id']], on='book_id').set_index('title')

In [ ]:
indices_map = id_map.set_index('id')

In [ ]:
def hybrid(userId, title): #id
    idx = indices[title]
    
    test = books[books['title'] == title]
    test1 = test.values.tolist()
    
    print(f'Title: {test1[0][6]}')
    print(f'Author: {test1[0][4]}')
    print(f'Release Date: {test1[0][5]} \n')

    sim_scores = list(enumerate(cosine_sim_content[int(idx)]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    book_indices = [i[0] for i in sim_scores]
    
    buks = books.iloc[book_indices][['title', 'authors', 'id', 'average_rating']]
    buks['est'] = buks['id'].apply(lambda x: svd.predict(userId, indices_map.loc[x]['book_id']).est)
    buks = buks.sort_values('est', ascending=False)
    sum = 0
    est = buks['est'].values.tolist()
    avg = buks['average_rating'].values.tolist()
    for i in range(len(est)):
        sum += abs(est[i] - avg[i])
    mae = sum/len(est)
    print(f'MAE: {round(mae, 3)}')
    newBuks = buks[buks['title'].notnull()]
    return newBuks.head(10)

In [ ]:
hybrid(1, "Harry Potter and the Sorcerer's Stone (Harry Potter, #1)")

Title: Harry Potter and the Sorcerer's Stone (Harry Potter, #1)
Author: J.K. Rowling, Mary GrandPré
Release Date: 1997.0 

MAE: 0.257


,title,authors,id,average_rating,est
3197,"The Concrete Blonde (Harry Bosch, #3; Harry Bo...",Michael Connelly,3198,4.12,4.823168
23,Harry Potter and the Goblet of Fire (Harry Pot...,"J.K. Rowling, Mary GrandPré",24,4.53,4.607536
22,Harry Potter and the Chamber of Secrets (Harry...,"J.K. Rowling, Mary GrandPré",23,4.37,4.564145
3752,"Harry Potter Collection (Harry Potter, #1-6)",J.K. Rowling,3753,4.73,4.476619
20,Harry Potter and the Order of the Phoenix (Har...,"J.K. Rowling, Mary GrandPré",21,4.46,4.381123
17,Harry Potter and the Prisoner of Azkaban (Harr...,"J.K. Rowling, Mary GrandPré, Rufus Beck",18,4.53,4.350748
2100,"The Harry Potter Collection 1-4 (Harry Potter,...","J.K. Rowling, Mary GrandPré",2101,4.66,4.327214
2750,"Trunk Music (Harry Bosch, #5; Harry Bosch Univ...",Michael Connelly,2751,4.15,4.327214
7017,The Magical Worlds of Harry Potter: A Treasury...,David Colbert,7018,3.96,4.327214
3735,Harry Potter Page to Screen: The Complete Film...,Bob McCabe,3736,4.56,4.327214


In [ ]:
hybrid(2, "Harry Potter and the Sorcerer's Stone (Harry Potter, #1)")

Title: Harry Potter and the Sorcerer's Stone (Harry Potter, #1)
Author: J.K. Rowling, Mary GrandPré
Release Date: 1997.0 

MAE: 0.311


,title,authors,id,average_rating,est
3197,"The Concrete Blonde (Harry Bosch, #3; Harry Bo...",Michael Connelly,3198,4.12,4.554038
20,Harry Potter and the Order of the Phoenix (Har...,"J.K. Rowling, Mary GrandPré",21,4.46,4.441863
17,Harry Potter and the Prisoner of Azkaban (Harr...,"J.K. Rowling, Mary GrandPré, Rufus Beck",18,4.53,4.441669
2496,"The Last Coyote (Harry Bosch, #4; Harry Bosch ...",Michael Connelly,2497,4.21,4.414830
23,Harry Potter and the Goblet of Fire (Harry Pot...,"J.K. Rowling, Mary GrandPré",24,4.53,4.325324
26,Harry Potter and the Half-Blood Prince (Harry ...,"J.K. Rowling, Mary GrandPré",27,4.54,4.252702
2100,"The Harry Potter Collection 1-4 (Harry Potter,...","J.K. Rowling, Mary GrandPré",2101,4.66,4.225755
2750,"Trunk Music (Harry Bosch, #5; Harry Bosch Univ...",Michael Connelly,2751,4.15,4.225755
7017,The Magical Worlds of Harry Potter: A Treasury...,David Colbert,7018,3.96,4.225755
3735,Harry Potter Page to Screen: The Complete Film...,Bob McCabe,3736,4.56,4.225755


In [ ]:
hybrid(50, "Harry Potter and the Sorcerer's Stone (Harry Potter, #1)")

Title: Harry Potter and the Sorcerer's Stone (Harry Potter, #1)
Author: J.K. Rowling, Mary GrandPré
Release Date: 1997.0 

MAE: 0.308


,title,authors,id,average_rating,est
3197,"The Concrete Blonde (Harry Bosch, #3; Harry Bo...",Michael Connelly,3198,4.12,4.421629
20,Harry Potter and the Order of the Phoenix (Har...,"J.K. Rowling, Mary GrandPré",21,4.46,4.360428
2496,"The Last Coyote (Harry Bosch, #4; Harry Bosch ...",Michael Connelly,2497,4.21,4.351806
17,Harry Potter and the Prisoner of Azkaban (Harr...,"J.K. Rowling, Mary GrandPré, Rufus Beck",18,4.53,4.347804
23,Harry Potter and the Goblet of Fire (Harry Pot...,"J.K. Rowling, Mary GrandPré",24,4.53,4.326885
3752,"Harry Potter Collection (Harry Potter, #1-6)",J.K. Rowling,3753,4.73,4.298571
22,Harry Potter and the Chamber of Secrets (Harry...,"J.K. Rowling, Mary GrandPré",23,4.37,4.285837
26,Harry Potter and the Half-Blood Prince (Harry ...,"J.K. Rowling, Mary GrandPré",27,4.54,4.230237
2000,Harry Potter: Film Wizardry,Brian Sibley,2001,4.48,4.161944
2750,"Trunk Music (Harry Bosch, #5; Harry Bosch Univ...",Michael Connelly,2751,4.15,4.161944


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=0b40164b-b106-44bf-9797-6a7361e30da7' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>